In [26]:
import pandas as pd
import numpy as np
import sys
import os 
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from category_encoders import OneHotEncoder

In [2]:
sys.path.append(os.path.abspath('../../util/model'))
from training import get_percent

In [3]:
info_data_path = "../../data/raw_data/data.info"
with open(info_data_path, 'r') as f:
    info = f.read().splitlines()
data_df1 = pd.read_parquet("../../data/merged_data/merged_data_v2_1.parquet")
data_df2 = pd.read_parquet("../../data/merged_data/merged_data_v2_2.parquet")
data_df3 = pd.read_parquet("../../data/merged_data/merged_data_v2_3.parquet")

In [4]:
print(data_df1.shape)
print(data_df2.shape)
print(data_df3.shape)


(5000000, 15)
(5000000, 15)
(1027106, 15)


In [5]:
raw_df =  pd.concat([data_df1, data_df2, data_df3])

In [6]:
def model_features_and_clean(df):
    '''Function to select features for modelling and clean the initial raw data. 
        Removes rows which contain NAN values in key_columns, and converts all NAN values to 0 for non_nan_cols

    :Parameters:
    ------------
        self.df: DataFrame
            DataFrame containing raw data

    :Returns:
    ---------
        self.df: DataFrame
            DataFrame containing features required for modelling
    '''

    model_features_list = ['transcript', 'position', 'nucleotides', 'reads_count', 'gene_id',
                            'dwellingtime_-1', 'std_-1', 'mean_-1',
                            'dwellingtime_0', 'std_0', 'mean_0',
                            'dwellingtime_+1', 'std_+1', 'mean_+1','label']
    non_nan_cols = []
    key_columns = ['label'] 

    df = df[model_features_list]
    df[non_nan_cols] = df[non_nan_cols].fillna(0)
    df = df.dropna(subset=key_columns)
    return df

In [7]:
cleaned_df = model_features_and_clean(raw_df)

In [8]:
# function to clean ==> added in np.min and np.max for all 
def feature_eng(df):
    temp = pd.DataFrame(df.groupby(['gene_id', 'transcript', 'position', 'nucleotides', 'reads_count', 'label'], as_index=False)
                           .agg({'dwellingtime_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'std_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'mean_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'dwellingtime_0': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'std_0': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'mean_0': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'dwellingtime_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'std_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max],
                                'mean_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean, np.min, np.max]}))
    temp.columns = ['gene_id', 'transcript', 'position', 'nucleotides', 'reads_count', 'label',
                        'dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean','dwelling_time_-1_min', 'dwelling_time_-1_max',
                        'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean','std_-1_min', 'std_-1_max',
                        'mean_-1_25', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean','mean_-1_min', 'mean_-1_max',
                        'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean','dwelling_time_0_min','dwelling_time_0_max',
                        'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean','std_0_min', 'std_0_max',
                        'mean_0_25', 'mean_0_50', 'mean_0_75', 'mean_0_mean','mean_0_min', 'mean_0_max',
                        'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean','dwelling_time_+1_min','dwelling_time_+1_max',
                        'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean','std_+1_min', 'std_+1_max',
                        'mean_+1_25', 'mean_+1_50', 'mean_+1_75', 'mean_+1_mean','mean_+1_min', 'mean_+1_max']
    return temp

In [9]:
percentiles_df = feature_eng(cleaned_df)

In [10]:
percentiles_df['position'] = percentiles_df['position'].astype(int)
# find the relative position of each read in each transcript
percentiles_df['relative_position'] = percentiles_df.groupby(['transcript','gene_id'])['position'].transform(lambda x: (x - x.min())/(x.max()-x.min()))
# note: have NAs because there's transcripts with only one position
# fill the NAs with 0
percentiles_df['relative_position'] = percentiles_df['relative_position'].fillna(0)

In [11]:
percentiles_df.shape

(121838, 61)

In [33]:
# split the nucleotides into seven columns
## this is just to show why after encoding some positions are missing
order_df = pd.DataFrame(percentiles_df['nucleotides'].str.split('').to_list())[[x for x in range(1, 8)]].rename(
    columns = {1: 'order_0', 2:'order_1', 
               3: 'order_2', 4:'order_3', 
               5: 'order_4', 6:'order_5',
               7: 'order_6'})
indiv_df = pd.concat([percentiles_df, order_df], axis = 1)
indiv_df.head()

,gene_id,transcript,position,nucleotides,reads_count,label,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,...,mean_+1_min,mean_+1_max,relative_position,order_0,order_1,order_2,order_3,order_4,order_5,order_6
0,ENSG00000000003,ENST00000373020,1006,TAGACCT,21,0,0.007650,0.009760,0.015100,0.010726,...,72.4,80.9,0.775510,T,A,G,A,C,C,T
1,ENSG00000000003,ENST00000373020,1013,AAAACTA,20,0,0.007965,0.009800,0.013175,0.011538,...,90.2,99.7,0.786499,A,A,A,A,C,T,A
2,ENSG00000000003,ENST00000373020,1149,GAAACAC,22,0,0.004917,0.007155,0.009473,0.008890,...,82.0,99.1,1.000000,G,A,A,A,C,A,C
3,ENSG00000000003,ENST00000373020,512,ATAACTC,20,0,0.004040,0.005990,0.008387,0.007247,...,83.5,94.5,0.000000,A,T,A,A,C,T,C
4,ENSG00000000003,ENST00000373020,689,TAAACAA,21,0,0.005310,0.007640,0.014300,0.009868,...,84.3,93.5,0.277865,T,A,A,A,C,A,A


In [34]:
columns = ['order_0', 'order_1', 'order_2', 'order_3', 'order_4', 'order_5', 'order_6']
for col in columns:
    print(f'{col}: {indiv_df[col].unique()}')

order_0: ['T' 'A' 'G' 'C']
order_1: ['A' 'T' 'G']
order_2: ['G' 'A']
order_3: ['A']
order_4: ['C']
order_5: ['C' 'T' 'A']
order_6: ['T' 'A' 'C' 'G']


In [17]:
X_raw = percentiles_df.drop(columns = ['label'], axis = 1)
y_raw = pd.DataFrame(percentiles_df['label'].astype(int))

In [28]:
def encoding_full_data(X_raw, y_raw):
    # new numeric cols
    numeric_cols =  ['reads_count','dwelling_time_-1_25',
                    'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean', 'dwelling_time_-1_min', 'dwelling_time_-1_max',
                    'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'std_-1_min', 'std_-1_max', 'mean_-1_25',
                    'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'mean_-1_min', 'mean_-1_max', 'dwelling_time_0_25',
                    'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',  'dwelling_time_0_min', 'dwelling_time_0_max',
                    'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean',  'std_0_min', 'std_0_max', 'mean_0_25',
                    'mean_0_50', 'mean_0_75', 'mean_0_mean','mean_0_min', 'mean_0_max', 'dwelling_time_+1_25',
                    'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean', 'dwelling_time_+1_min', 'dwelling_time_+1_max',
                    'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean','std_+1_min', 'std_+1_max', 'mean_+1_25',
                    'mean_+1_50', 'mean_+1_75', 'mean_+1_mean', 'mean_+1_min','mean_+1_max', 'relative_position']
    one_hot_col = ['position_0', 'position_1', 'position_2', 'position_3', 'position_4', 'position_5', 'position_6']

    # piping the encoding
    numeric_encoder = Pipeline([('scale', StandardScaler())])
    one_hot_encoder = Pipeline([('one_hot_enocde', OneHotEncoder(handle_unknown='ignore'))])
    preprocessor = ColumnTransformer(transformers=[("num", numeric_encoder, numeric_cols), 
                                                    ("one_hot_encode", one_hot_encoder, one_hot_col)], remainder='passthrough')

    # getting list of column names to map
    for i in range(7):
        X_raw['position_' + str(i)] = X_raw['nucleotides'].apply(lambda x: x[i])
    ref_df = X_raw # df used as reference for encoding
    columns_to_map = numeric_cols
    one_hot = OneHotEncoder(handle_unknown='ignore',use_cat_names=True)
    x_df_ohe = pd.DataFrame(one_hot.fit_transform(ref_df[one_hot_col]))
    x_df_ohe.columns = one_hot.get_feature_names()
    columns_to_map = numeric_cols + list(x_df_ohe.columns)

    print('columns after preprocessing :', columns_to_map,  '\n')

    # applying encoding on columns in df and creating pipeline
    X_raw_enc = pd.DataFrame({col: vals for vals, col in zip(preprocessor.fit_transform(ref_df, y_raw).T, columns_to_map)})
    pipe = Pipeline(steps=[("preprocessor", preprocessor)])
    pipe = pipe.fit(ref_df, y_raw)

    return X_raw_enc, pipe, columns_to_map

In [29]:
X_raw_enc, pipe, columns_to_map = encoding_full_data(X_raw, y_raw)

columns after preprocessing : ['reads_count', 'dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean', 'dwelling_time_-1_min', 'dwelling_time_-1_max', 'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'std_-1_min', 'std_-1_max', 'mean_-1_25', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'mean_-1_min', 'mean_-1_max', 'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean', 'dwelling_time_0_min', 'dwelling_time_0_max', 'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean', 'std_0_min', 'std_0_max', 'mean_0_25', 'mean_0_50', 'mean_0_75', 'mean_0_mean', 'mean_0_min', 'mean_0_max', 'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean', 'dwelling_time_+1_min', 'dwelling_time_+1_max', 'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'std_+1_min', 'std_+1_max', 'mean_+1_25', 'mean_+1_50', 'mean_+1_75', 'mean_+1_mean', 'mean_+1_min', 'mean_+1_max', 'relative_position', 'position_0_T',

In [30]:
X_raw_enc.head()

,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,dwelling_time_-1_min,dwelling_time_-1_max,std_-1_25,std_-1_50,std_-1_75,...,position_2_A,position_3_A,position_4_C,position_5_C,position_5_T,position_5_A,position_6_T,position_6_A,position_6_C,position_6_G
0,-0.506116,2.671903,1.730975,2.008774,1.465633,1.425162,-0.715276,-0.289779,-0.502084,-0.712691,...,0,1,1,1,0,0,1,0,0,0
1,-0.513397,2.954481,1.754884,1.23072,1.920001,0.016588,1.005682,-0.722286,-0.818518,-0.965746,...,1,1,1,0,1,0,0,1,0,0
2,-0.498834,0.220651,0.173918,-0.26577,0.437666,0.731387,2.288764,-0.711435,-0.536694,-0.313842,...,1,1,1,0,0,1,0,0,1,0
3,-0.513397,-0.56653,-0.522423,-0.70431,-0.481384,0.731387,-0.959673,-1.002874,-0.885265,-0.977201,...,1,1,1,0,1,0,0,0,1,0
4,-0.506116,0.572752,0.463812,1.685427,0.985092,0.016588,0.160478,-0.996673,-1.055843,-1.175063,...,1,1,1,0,0,1,0,1,0,0


In [35]:
X_raw_enc.shape

(121838, 74)

In [38]:
%cd ../../data/raw_data

/Users/xinrantao/Desktop/DSA4262-ACMXZ/data/raw_data


In [39]:
## raw data for eda
X_raw_enc.to_parquet("X_raw_enc.parquet") 
y_raw.to_parquet("y_raw.parquet")